In [ ]:
!pip install tflite-support
!pip install flatbuffers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 7.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install keras

In [ ]:
pip install tensorflow

In [ ]:
import pandas as pd

# Load data dari file CSV
data_path = '/content/drive/My Drive/KalorikuApp/nutrition.csv'  # Pastikan path sesuai dengan letak file CSV
nutrition_data = pd.read_csv(data_path)

# Tampilkan beberapa baris pertama untuk verifikasi
print(nutrition_data.head())


   id  calories  proteins   fat  carbohydrate                name  \
0   1     280.0       9.2  28.4           0.0                Abon   
1   2     513.0      23.7  37.0          21.3        Abon haruwan   
2   3       0.0       0.0   0.2           0.0           Agar-agar   
3   4      45.0       1.1   0.4          10.8  Akar tonjong segar   
4   5      37.0       4.4   0.5           3.8       Aletoge segar   

                                               image  
0  https://img-cdn.medkomtek.com/PbrY9X3ignQ8sVuj...  
1  https://img-global.cpcdn.com/recipes/cbf330fbd...  
2  https://res.cloudinary.com/dk0z4ums3/image/upl...  
3  https://images.tokopedia.net/img/cache/200-squ...  
4  https://nilaigizi.com/assets/images/produk/pro...  


In [ ]:
# Contoh preprocessing, asumsikan 'calories' sebagai salah satu fitur
nutrition_data['calories'] = pd.to_numeric(nutrition_data['calories'], errors='coerce')
nutrition_data.dropna(subset=['calories'], inplace=True)  # Hapus baris dengan missing value


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load data
data_path = '/content/drive/My Drive/KalorikuApp/nutrition.csv'
nutrition_data = pd.read_csv(data_path)

# Clean data
nutrition_data.dropna(subset=['calories', 'proteins', 'fat', 'carbohydrate'], inplace=True)

# Tambahkan Fitur and Target
features = nutrition_data[['proteins', 'fat', 'carbohydrate']]
target = nutrition_data['calories']

# Preprocessing
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Membagi dataset untuk training and testing
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load data
data_path = '/content/drive/My Drive/KalorikuApp/nutrition.csv'
nutrition_data = pd.read_csv(data_path)
nutrition_data['calories'] = pd.to_numeric(nutrition_data['calories'], errors='coerce')
nutrition_data.dropna(subset=['proteins', 'fat', 'carbohydrate', 'calories'], inplace=True)

# Preprocessing
scaler = StandardScaler()
features = scaler.fit_transform(nutrition_data[['proteins', 'fat', 'carbohydrate']].values)
target = nutrition_data['calories'].values

# Pembuatan Model
model = Sequential([
    Dense(128, activation='relu', input_shape=(3,)),  # 3 features: proteins, fat, carbohydrate
    Dense(64, activation='relu'),
    Dense(1, activation='linear')  # Output layer for predicting calories
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
model.summary()

# Train model yang telah dibuat
model.fit(features, target, epochs=10, batch_size=10, validation_split=0.2)


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 128)               512       
                                                                 
 dense_28 (Dense)            (None, 64)                8256      
                                                                 
 dense_29 (Dense)            (None, 1)                 65        
                                                                 
Total params: 8833 (34.50 KB)
Trainable params: 8833 (34.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
108/108 [==============================] - 3s 8ms/step - loss: 69530.9297 - mean_absolute_error: 203.7726 - val_loss: 39452.5625 - val_mean_absolute_error: 153.7071
Epoch 2/10
108/108 [==============================] - 0s 4ms/step - loss: 37657.5820 - mean_absolute_error: 

In [ ]:
import pandas as pd

def calculate_daily_calories(weight, height, age, goal_index):
    # Asumsikan gender sebagai laki laki untuk mempermudah
    BMR = 10 * weight + 6.25 * height - 5 * age + 5  # Plus 5 untuk laki laki, kurangi 161 untuk perempuan

    # Sesuaikan BMR berdasarkan goal_index
    if goal_index == 0:  # Menaikkan berat badan
        return BMR + 500
    elif goal_index == 1:  # Menurunkan berat badan
        return BMR - 500
    else:  # Menjaga berat badan
        return BMR


def recommend_meals(data, daily_calories):
    meal_calories = daily_calories / 3
    suitable_foods = data[(data['calories'] >= 0.8 * meal_calories) & (data['calories'] <= 1.2 * meal_calories)]
    if suitable_foods.shape[0] >= 3:
        return suitable_foods.sample(3)
    else:
        print("Not enough suitable meals found to recommend.")
        return pd.DataFrame()  # Return DataFrame kosong jika kurang dari 3 makanan yang cocok ditemukan

def main():
    data_path = '/content/drive/My Drive/KalorikuApp/nutrition.csv'
    nutrition_data = pd.read_csv(data_path)
    nutrition_data['calories'] = pd.to_numeric(nutrition_data['calories'], errors='coerce')
    nutrition_data.dropna(subset=['calories'], inplace=True)

    # Contoh input dari User
    weight = float(input("Enter your weight in kg: "))
    height = float(input("Enter your height in cm: "))
    age = int(input("Enter your age in years: "))

    # Konversikan goal dari 'string' menjadi 'float'
    goals = {"increase": 0.0, "decrease": 1.0, "maintain": 2.0}
    goal_input = input("Enter your goal (increase, decrease, maintain): ").lower()
    if goal_input in goals:
        goal_index = goals[goal_input]
    else:
        print("Invalid goal entered. Please enter 'increase', 'decrease', or 'maintain'.")
        return

    # Hitung kalori harian
    daily_calories_needed = calculate_daily_calories(weight, height, age, goal_index)

    # Dapatkan rekomendasi makanan
    recommended_meals = recommend_meals(nutrition_data, daily_calories_needed)

    if not recommended_meals.empty:
        # Ekstrak dalam variabel terpisah
        meal_1 = recommended_meals.iloc[0]
        meal_2 = recommended_meals.iloc[1]
        meal_3 = recommended_meals.iloc[2]

        print("Meal 1:", meal_1)
        print("Meal 2:", meal_2)
        print("Meal 3:", meal_3)
    else:
        print("Unable to extract meals as the recommendation list is empty.")

if __name__ == "__main__":
    main()


Enter your weight in kg: 50
Enter your height in cm: 180
Enter your age in years: 90
Enter your goal (increase, decrease, maintain): maintain
Meal 1: id                                                           1266
calories                                                    353.0
proteins                                                      2.9
fat                                                           0.5
carbohydrate                                                 84.4
name                                                 Tepung Sukun
image           https://www.static-src.com/wcsstore/Indraprast...
Name: 1265, dtype: object
Meal 2: id                                                           1257
calories                                                    355.0
proteins                                                      9.2
fat                                                           3.9
carbohydrate                                                 73.7
name                    

In [ ]:
model_save_path = '/content/drive/My Drive/KalorikuApp/my_model.h5'


In [ ]:
model.save(model_save_path)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf

# Mount Google Drive (hanya untuk keperluan colab)
from google.colab import drive
drive.mount('/content/drive')

# Sesuaikan path di mana model TensorFlow Lite akan disimpan
tflite_model_path = '/content/drive/My Drive/KalorikuApp/metadata.tflite'

# Konversikan menjadi format TFlite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model TFlite
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print("Model saved to Google Drive successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model saved to Google Drive successfully!


In [ ]:
import tensorflow as tf

# Sesuaikan dengan path dimana file TFlite berada
tflite_model_path = '/content/drive/My Drive/KalorikuApp/metadata.tflite'
try:
    # Load model TFlite sebagai interpreter
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()
    print("The model is loaded successfully and does not seem to be corrupt.")
except Exception as e:
    print("Failed to load the model:", e)


The model is loaded successfully and does not seem to be corrupt.


Convert tflite model to tfjs

In [ ]:
!tensorflowjs_converter \
  --input_format=tf_lite \
  --output_format=tfjs_graph_model \
  --quantization_bytes=1 \
  /content/drive/My Drive/KalorikuApp/metadata.tflite \
  /content/drive/My Drive/KalorikuApp/metadata_tfjs


2024-06-18 11:50:04.654993: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 11:50:05.921205: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: TensorFlow.js model converters. [-h]
                                       [--input_format {tf_hub,tf_frozen_model,tf_saved_model,tfjs_layers_model,keras,keras_keras,keras_saved_model}]
                                       [--output_format {tfjs_graph_model,tfjs_layers_model,keras,keras_keras,keras_saved_model}]
                                       [--signature_name SIGNATURE_NAME]
                                       [--saved_model_tags SAVED_MODEL_TAGS]
                                       [--quantize_float16 [QUANTIZE_FLOAT16]]
           

In [ ]:
import numpy as np
import tensorflow as tf

# Load model TFlite
tflite_model_path = '/content/drive/My Drive/KalorikuApp/metadata.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Dapatkan input dan output tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Input uji sampel (ganti dengan data input asli)
test_input = np.array([[2.0, 0.5, 1.2]], dtype=np.float32)  # Contoh values dari proteins, fat, carbohydrate

# Atur input tensor
interpreter.set_tensor(input_details[0]['index'], test_input)

# Run
interpreter.invoke()

# Dapatkan output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
print(f'Predicted Calories: {output_data[0][0]}')


Predicted Calories: 415.25274658203125
